# Desenvolvimento de codigo
Apos o recebmento dos arquivos CONLL, foi feito a separação dos arquivos em suas determinadas revistas, em seguida foram separados os arquivos para as pastas separadas por areas de conhecimento. Apos esse processo foi iniciado o processo de codificação de uma atraves da plataforma JUPYTER para extrair as informações em arquivos ais facilmentes editaveis. Para isso forom utiizadas as segintes bibliotecas:

In [18]:
import numpy as np #Biblioteca de suporte de arrays e matriz multidimensionais
import pandas as pd #Biblioteca de manipulação e análise de dado
import os #Esse modulo prove alguns metodos para atuar com o sistema operacional, nesse projeto utilizado para gerenciamento de diretorios
import sys #Esse modulo prove acesso à algumas variaveis de sistema, nesse projeto utilizado para gerenciamento de diretorios
import matplotlib.pyplot as plt #Biblioteca padrão pala plotagem de graficos
import seaborn as sns #Biblioteca para melhoria de interface dos graficos

# Desenvolvimento de funções.
    Em seguida foi desenvolvido o codigo necessario para a transformação dos arquivos CONLL em arquivos com dados analisaeis, seguindo as orientções de requisitos.

## getListCount
    Essa função recebe uma matriz de dados(df) e retorna uma lista com a contagem de inicidencia de uma coluna especifica(title).
    Utilizada para fazer a contagem dos frames

In [19]:
def getListCount(df,title):
    listOfterms = df[title].value_counts()
    return listOfterms[1:]

# returnHeaderAndContent
    Essa função recebe uma matriz de dados (df) retorna duas matrizes, uma com todos os dados referentes a primeira sentença  e outro com o restante do conteudo da matriz
    Essa função é executada duas vezes, uma vez para separar o titulo da revista e o resumo todo e em seguida o titulo do resumo e o conteudo

In [20]:
def returnHeaderAndContent(df):
    for index, row in df.iterrows():
        if(index != row['ID']-1):
            index_title = index
            break
    temp = index_title
    count = temp
    while temp == count:
        if(df.iloc[temp-1]['FORM'] != df.iloc[index_title-1]['FORM']):
            break
        count = count + index_title
        temp  = temp + index_title
    return  df.iloc[:temp-index_title].reset_index(drop=True), df.iloc[temp-index_title:].reset_index(drop=True)

## getPlot
Essa função recebe uma matriz de dados(df), um nome de uma coluna(label),e o nome do arquivo a ser salvo(filename).
Ele gera um grafico dos dados da matriz e salva esse grafico com o nome especificados.

In [21]:
def getPlot(df,label,fileName):
    sns.set(style="whitegrid")
    fig = plt.figure(figsize=(23, 10))
    fig.subplots_adjust(bottom=0.4)
    plt.xticks(rotation=70)
    try:
        ax = sns.barplot(x="index", y=label, data=df)
        fig.savefig(fileName)
    except:
        print("")
        
    plt.close(fig)

# Processo de extração dos dados
## Listagem e criação de pastas
    Utilizando a pasta do arquivo atual como base, inicialmente são listados todos os arquivos dessa pasta e adicionado na variavel directories.

In [22]:
path = "."
directories = os.listdir( path )

## Remoção de pastas não utilizadas
    São retirados os arquivos e diretorios que não são relevantes para a analise.

In [23]:
def cleanDirectory():
    try:
        for directory in directories:
            if(directory.startswith(".") or directory.endswith(".ipynb") or directory.startswith("_")):
                directories.remove(directory)
                cleanDirectory()
    except:
        print("")
cleanDirectory()

In [24]:
directories

['milbank-quarterly',
 'quality-&-safety',
 'global-change-biology-bioenergy',
 'advances-in-agronomy',
 'academic-medicine',
 'theoretical-and-applied-genetics',
 'agronomy-for-sustainable-development',
 'value-in-health',
 'agricultural-and-forest-meteorology',
 'health-affairs']

# Extração de dados

    É criado um laço de repetição passando por todos os diretorios do diretorio base e listado todos os arquivos desse diretorio que possuem a extensão CONLL.
    Dessa nova lista tambem é criado um novo laço de repetição para cada arquivo gera uma matriz que possue todos os dados do arquivo CONLL utilizando a função returnHeaderAndContent é possivel separar o titulo do journal do conteudo do resumo. Apos isso utilizando novamente a função returnHeaderAndContent é possivel separar o titulo do resumo do seu conteudo e separalos em matrizes diferentes.

In [25]:
columns = ['ID', 'FORM' ,'LEMMA' ,'PLEMMA' ,'POS' ,'PPOS' ,'FEAT' ,'PFEAT' ,'HEAD' ,'PHEAD' ,'DEPREL' ,'PDEPREL','FILLPRED','PRED','APRED']
df_all_sum = pd.DataFrame(columns=columns)
df_head_sum = pd.DataFrame(columns=columns)
df_content_sum = pd.DataFrame(columns=columns)
df_title_sum  = pd.DataFrame(columns=columns)
df_tempo  = pd.DataFrame(columns=columns)
for directory in directories:
    columns = ['ID', 'FORM' ,'LEMMA' ,'PLEMMA' ,'POS' ,'PPOS' ,
       'FEAT' ,'PFEAT' ,'HEAD' ,'PHEAD' ,'DEPREL' ,'PDEPREL','FILLPRED','PRED','APRED']
    listOfFiles = []
    folderPath = ".//{}".format(directory)
    files = os.listdir( folderPath )
    for fl in files:

        if(fl.endswith('.conll')):
            listOfFiles.append("{}//{}".format(folderPath,fl))
    for i in listOfFiles:
        df_tempo  = pd.DataFrame(columns=columns)
        df_all = pd.read_csv(i,sep="\t",names=columns)
        df_title, df_temp = returnHeaderAndContent(df_all)
        df_head, df_content = returnHeaderAndContent(df_temp)
        df_title_sum =   df_title_sum.append(df_title)
        df_tempo = df_head.append(df_content)
        df_all_sum =     df_all_sum.append(df_tempo) 
        df_head_sum =    df_head_sum.append(df_head) 
        df_content_sum = df_content_sum.append(df_content) 

Utilizando essas matrizes recem criadas é utilizada a função getListCount para obter a contagem dos frames.

In [26]:
list_all = getListCount(df_all_sum,'PRED')[1:].to_frame().reset_index()
list_head  = getListCount(df_head_sum,'PRED')[1:].to_frame().reset_index()
list_content = getListCount(df_content_sum,'PRED')[1:].to_frame().reset_index()

Apos a criação da lista de abstracts foi utilizada a função da biblioteca do pandas to_excel para criar o documento de cada uma das listas. E finalizando tambem foi criado os graficos de cada uma das listas.

In [27]:
list_all.to_excel("_list_all.xls", index = None, header=True)
getPlot(list_all,"PRED","_all.png")
list_head.to_excel("_list_head.xls", index = None, header=True)
getPlot(list_head,"PRED","_head.png")
list_content.to_excel("_list_content.xls", index = None, header=True)
getPlot(list_content,"PRED","_content.png")